<html><h1 style="background: -webkit-gradient(linear,left top,left bottom,from(#fff),to(#efefef)); border-radius: 10px; box-shadow: 0 0 4px;
width: 100%; border: 1px solid grey; padding: 1em; box-sizing: border-box; margin-top: 5px; margin-bottom: 5px; text-align: center; background: brown; color: white; letter-spacing: 2px; font-weight: bold;line-height: 48px;">Integer Triangles Trait Cards<br>Computed with Julia<br><a href="http://luschny.de/julia/triangles/TutorialTrianglesPart1.html">(See also the Tutorial)</a></h1>

In [ ]:
# Pkg.add(PackageSpec(url="https://github.com/OpenLibMathSeq/IntegerTriangles.jl"))
# Pkg.add("IntegerTriangles.jl"))

In [ ]:
using IntegerTriangles

Definition: A *integer triangle* is an array of arrays whose members are integers. 
It has the type **ℤTri**.

An integer is a multiple precision integer which is created by the constructor **ℤInt**.

The row of an integer triangle is an integer sequence, **ℤSeq**.

Examples for the creation of an integer triangle:

In [ ]:
[[ℤInt(k) for k in 0:n] for n in 0:6]

In [ ]:
ℤSeq(7, n -> ℤInt(n))

In [ ]:
T = Telescope(6, n -> ℤInt(n)) 
Println.(T)

The shape of the triangle is /not/ fixed. It includes the cases denoted by the OEIS keywords *tabl* and *tabf*. We allow the empty sequence to be element of an integer triangle.

In [ ]:
function Divisors(n) 
    n == 0 && return ℤSeq()
    [ℤInt(d) for d in 1:n if rem(n, d) == 0]
end

In [ ]:
ℤTri(9, Divisors)

In [ ]:
D = ℤTri(9, Divisors)
isa(D, ℤTri)

In [ ]:
import Base.sum
sum(T::ℤTri) = sum.(T)

In [ ]:
sum(Divisors.(0:7))

In [ ]:
S = ℤSeq(5) 

In [ ]:
T = ℤTri(5) 

If the keyword 'reg' is set to true, the constructor returns an uninitialized regular triangle.

In [ ]:
T = ℤTri(6, reg=true) 

In [ ]:
for n in 1:6 T[n] = [ℤInt(k) for k in 1:n] end
T

In [ ]:
T[3]

In [ ]:
T[3][2]

In [ ]:
isa(T, ℤTri) |> println
isa(T[3], ℤSeq) |> println
isa(T[3][2], ℤInt) |> println

In [ ]:
function LahIndexed(n, k)
    function recLah(n, k)
        k  < 0 && return ℤInt(0)
        k == n && return ℤInt(1)
        recLah(n-1, k-1) + recLah(n-1, k)*(n+k-1)
    end
    recLah(n, k)
end

[[LahIndexed(n, k) for k in 0:n] for n in 0:6]

Thus LahNumbers(n) returns an n-element Array{Array{ℤInt,1},1}
which is according to our definiton a triangle with *n* rows.

In [ ]:
const CacheLah = Dict{Int, ℤSeq}([0 => [ℤInt(1)]])

function LahNumbers(n)
    haskey(CacheLah, n) && return CacheLah[n]
    prevrow = LahNumbers(n-1)
    row = ℤSeq(n+1)
    row[1] = 0; row[n+1] = 1
    for k in 2:n
        row[k] = prevrow[k-1] + prevrow[k]*(n+k-2)    
    end
    CacheLah[n] = row
end

In [ ]:
LahNumbers(5)

In [ ]:
println(CacheLah)

Let's check the time and space consumtion: 

In [ ]:
@time LahNumbers(100);

In [ ]:
LahNumbers(n, k) = LahNumbers(n+1)[k+1]
LahNumbers(5, 3)

In [ ]:
methods(LahNumbers)

In [ ]:
function LahTriangle(size) 
    length(CacheLah) < size && LahNumbers(size)
    [CacheLah[n] for n in 0:size-1] 
end

In [ ]:
methods(LahTriangle)

In [ ]:
T = LahTriangle(7)
length(T)

In [ ]:
evensum(A) = sum(A[1:2:end]) 
oddsum(A)  = sum(A[2:2:end])
altsum(A)  = evensum(A) - oddsum(A)
evensum(T::ℤTri) = evensum.(T)
oddsum(T::ℤTri)  = oddsum.(T)
altsum(T::ℤTri)  = evensum(T) - oddsum(T)

In [ ]:
T = LahTriangle(10)
Println.([sum(T), evensum(T), oddsum(T), altsum(T)]);

In [ ]:
middle(A) = A[div(end + 1, 2)] 
middle(T::ℤTri) = middle.(T)

middle(LahTriangle(10)) |> Println

In [ ]:
central(T::ℤTri) = middle.(T[1:2:end])

central(LahTriangle(16)) |> Println

In [ ]:
function DiagonalTriangle(T::ℤTri)
    dim = length(T)
    U = ℤTri(dim)
    for n in 1:dim
        R = ℤSeq(div(n+1,2))
        for k in 0:div(n-1,2)
            R[k+1] = T[n-k][k+1]
        end
        U[n] = R
    end
    U
end

In [ ]:
T = LahTriangle(10) 
DiagonalTriangle(T)

In [ ]:
diagsum(T) = sum(diag(T))
diagsum(LahTriangle(9))

In [ ]:
leftside(A)  = A[1] 
rightside(A) = A[end] 

rightside(T::ℤTri) = rightside.(T)
leftside( T::ℤTri) = leftside.(T)

In [ ]:
function profile(T::ℤTri)
    println("Triangle: ");
    for row in T Println(row) end; println()
    print("Sum:      "); sum(T)       |> Println 
    print("EvenSum:  "); evensum(T)   |> Println 
    print("OddSum:   "); oddsum(T)    |> Println 
    print("AltSum:   "); altsum(T)    |> Println 
    print("DiagSum:  "); diagsum(T)   |> Println 
    print("Middle:   "); middle(T)    |> Println 
    print("Central:  "); central(T)   |> Println 
    print("LeftSide: "); leftside(T)  |> Println 
    print("RightSide:"); rightside(T) |> Println 
end

In [ ]:
profile(LahTriangle(10))

In [ ]:
invT = InverseTriangle(T)      
isa(invT, ℤTri) |> println
invT

In [ ]:
Apery = [
            ℤInt(1),
            5,
            73,
            1445,
            33001,
            819005,
            21460825,
            584307365
        ]

BinomialTransform(Apery) |> Println

In [ ]:
dim = 8
TraitCard(NarayanaTriangle, NarayanaTransform, dim)

In [ ]:
TraitCard(LaguerreTriangle, LaguerreTransform, dim)

In [ ]:
TraitCard(MotzkinTriangle, MotzkinTransform, dim)

In [ ]:
TraitCard(JacobsthalTriangle, JacobsthalTransform, dim)